In [1]:
import open3d as o3d
import numpy as np
import mcubes
import trimesh
from jnerf.runner import Runner 
from jnerf.utils.config import init_cfg
init_cfg('./projects/ngp/configs/ngp_base.py')
runner = Runner()
result = runner.mesh()
result.show()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[i 0907 10:07:23.799517 44 lock.py:85] Create lock file:/home/featurize/.cache/jittor/jt1.3.4/g++9.3.0/py3.7.10/Linux-5.4.0-91xc3/IntelRXeonRGolx46/jittor.lock
[i 0907 10:07:23.818187 44 compiler.py:955] Jittor(1.3.4.17) src: /home/featurize/work/.local/lib/python3.7/site-packages/jittor
[i 0907 10:07:23.834343 44 compiler.py:956] g++ at /usr/bin/g++(9.3.0)
[i 0907 10:07:23.834576 44 compiler.py:957] cache_path: /home/featurize/.cache/jittor/jt1.3.4/g++9.3.0/py3.7.10/Linux-5.4.0-91xc3/IntelRXeonRGolx46/default
[i 0907 10:07:23.866258 44 __init__.py:411] Found nvcc(11.2.152) at /usr/local/cuda/bin/nvcc.
[i 0907 10:07:23.891645 44 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.
[i 0907 10:07:24.072649 44 compiler.py:1010] cuda key:cu11.2.152_sm_86
[i 0907 10:07:24.151910 44 compiler.py:34] Create cache dir: 

[i 0907 10:07:32.109403 60 log.cc:351] Load log_sync: 1
[i 0907 10:07:49.612548 60 __init__.py:227] Total mem: 55.60GB, using 16 procs for compiling.


[i 0907 10:08:40.421226 60 log.cc:351] Load log_sync: 1
[i 0907 10:08:40.423296 60 jit_compiler.cc:28] Load cc_path: /usr/bin/g++


Compiling jittor_core(147/147) used: 50.238s eta: 0.000s


[i 0907 10:08:40.725501 60 init.cc:62] Found cuda archs: [86,]
[i 0907 10:08:41.919880 60 compile_extern.py:516] mpicc not found, distribution disabled.
[i 0907 10:08:41.922383 60 compiler.py:34] Create cache dir: /home/featurize/.cache/jittor/cutt
[i 0907 10:08:41.923542 60 compile_extern.py:333] Downloading cutt...


...

100%|██████████| 400/400 [00:19<00:00, 20.92it/s]


load val data


  0%|          | 0/20 [00:00<?, ?it/s]
Compiling Operators(2/6) used: 3.31s eta: 6.62s 5/6) used: 4.32s eta: 0.865s 6/6) used: 6.34s eta:    0s 
100%|██████████| 20/20 [00:09<00:00,  2.11it/s]


Loading ckpt from: ./logs/lego/params.pkl



Compiling Operators(3/19) used: 7.32s eta:   39s 8/19) used: 8.33s eta: 11.4s 12/19) used: 9.33s eta: 5.44s 16/19) used: 10.3s eta: 1.94s 19/19) used: 12.4s eta:    0s 

Compiling Operators(1/1) used: 2.86s eta:    0s 

Compiling Operators(2/15) used: 6.32s eta: 41.1s 9/15) used: 7.33s eta: 4.89s 14/15) used: 8.34s eta: 0.596s 15/15) used: 9.35s eta:    0s 
